<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления 

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

# Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [1]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

--2021-02-28 19:40:27--  https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1453905423 (1.4G) [application/octet-stream]
Saving to: ‘SO_vectors_200.bin?download=1.1’

SO_vectors_200.bin? 100%[===================>]   1.35G  8.45MB/s    in 1m 54s  

2021-02-28 19:42:21 (12.2 MB/s) - ‘SO_vectors_200.bin?download=1.1’ saved [1453905423/1453905423]



In [2]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin?download=1", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [3]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [4]:
print(f"Num of words: {len(wv_embeddings.index2word)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слов `cat` топ-5 близких слов к слову `dog`? Какое место? 

In [5]:
# method most_simmilar
wv_embeddings.most_similar("dog",negative=None, topn=30)

[('animal', 0.8564180135726929),
 ('dogs', 0.7880867123603821),
 ('mammal', 0.7623804807662964),
 ('cats', 0.7621253728866577),
 ('animals', 0.760793924331665),
 ('feline', 0.7392398118972778),
 ('bird', 0.7315489053726196),
 ('animal1', 0.7219215631484985),
 ('doggy', 0.7213349938392639),
 ('labrador', 0.7209131717681885),
 ('canine', 0.7209056615829468),
 ('meow', 0.7185295820236206),
 ('cow', 0.7080444693565369),
 ('dog2', 0.7057910561561584),
 ('woof', 0.7050611972808838),
 ('dog1', 0.7038840055465698),
 ('dog3', 0.701882004737854),
 ('penguin', 0.6970292329788208),
 ('bulldog', 0.6940488815307617),
 ('mammals', 0.6931389570236206),
 ('bark', 0.6913799047470093),
 ('fruit', 0.6892251968383789),
 ('reptile', 0.6891210079193115),
 ('furry', 0.6863498687744141),
 ('carnivore', 0.6862949728965759),
 ('cat', 0.6852341294288635),
 ('horse', 0.6833381056785583),
 ('kitten', 0.6820152997970581),
 ('sheep', 0.6802570223808289),
 ('chihuahua', 0.6791757941246033)]

Нет, не входит, 26 место.

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [6]:
import numpy as np
import re
# you can use your tokenizer
# for example, from nltk.tokenize import WordPunctTokenizer
class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        return re.findall('\w+', text)
tokenizer = MyTokenizer()

In [7]:
tokenizer.tokenize("23 java c# sd")

['23', 'java', 'c', 'sd']

In [8]:
from nltk.tokenize import WordPunctTokenizer
s = "23 java c# sd"
a = WordPunctTokenizer().tokenize(s)
print(a)

['23', 'java', 'c', '#', 'sd']


In [9]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении
        
        return: векторное представление для вопроса
    """
    
    vector = [embeddings[word.lower()] for word in tokenizer.tokenize(question) if word.lower() in embeddings]
    if (len(vector) == 0):
          vector = np.array([0]*dim)
          return vector
    return np.mean(vector, axis=0)

Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [10]:
question = "I love neural networks"
question_to_vec(question, wv_embeddings, tokenizer, dim=200)

array([-1.0142275 , -1.6891261 , -1.2854122 , -1.3710302 ,  0.15916634,
        0.06706961,  1.1188413 , -0.07051762,  0.5386369 , -0.37139115,
        0.28913018,  1.03125   , -2.865466  , -1.3192873 ,  2.2098296 ,
        1.1678672 ,  1.0841951 ,  1.8467226 , -0.98733807,  0.8720946 ,
       -0.17219625,  0.73839015, -0.7827826 , -0.8827869 , -0.6152611 ,
       -0.15872358, -0.6170798 ,  0.36668167,  2.0137532 ,  0.2627472 ,
       -1.4950123 , -1.386715  , -1.2049937 , -1.016617  ,  2.7621174 ,
        1.7255713 , -0.19038178, -0.5899447 , -0.1141155 , -1.8890352 ,
       -1.7883505 , -0.20420384, -0.7062034 ,  2.489034  , -0.15452717,
        1.1255013 ,  0.2610835 ,  1.3153247 , -0.57667524, -0.3327141 ,
        1.6764902 ,  0.91229993, -0.69880134,  2.3752553 , -1.7634224 ,
        3.1378052 ,  1.2243204 , -1.0554415 ,  0.7897571 ,  0.04404024,
        1.1623045 , -0.4478023 , -0.9149816 ,  0.01501505, -0.85523087,
       -1.0183454 , -0.7162116 ,  1.0727204 ,  0.8881876 , -0.29

**-1.29**


### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями. 

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv 
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`? 
1

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера. 
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

$\text{DCG@10} = \frac{1}{\log_2(1+9)}\cdot[9 \le 10] = \frac{1}{\log_2(10)} \text{ ~ } 0.3$

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [11]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """
    N = len(dup_ranks) 
    hits_value = sum([1 for ranks in dup_ranks if ranks <= k])/N
    return hits_value    

In [12]:
hits_count([2, 3, 4], 3)

0.6666666666666666

In [13]:
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """
    N = len(dup_ranks) 
    dcg_value = sum([(1/np.log2(1+ranks)) for ranks in dup_ranks if ranks <= k])/N
    return dcg_value

In [14]:
dcg_score([9], 10)

0.3010299956639812

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [15]:
import pandas as pd

In [16]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наши кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [[candidates_ranking[k].index(elements) + 1 for elements in copy_answers] for k in range(len(candidates_ranking))][0]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [17]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
!unzip -o /content/drive/MyDrive/stackoverflow_similar_questions.zip

Archive:  /content/drive/MyDrive/stackoverflow_similar_questions.zip
  inflating: data/.DS_Store          
  inflating: __MACOSX/data/._.DS_Store  
  inflating: data/train.tsv          
  inflating: data/validation.tsv     


Считайте данные.

In [20]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        data.append(line.split('\t'))
    return data

Нам понадобиться только файл validation.

In [21]:
validation_data = read_corpus('./data/validation.tsv')

Кол-во строк

In [22]:
len(validation_data)

3760

In [23]:
validation_data[0][:10]

['How to print a binary heap tree without recursion?',
 'How do you best convert a recursive function to an iterative one?',
 'How can i use ng-model with directive in angular js',
 'flash: drawing and erasing',
 'toggle react component using hide show classname',
 'Use a usercontrol from another project to current webpage',
 '~ Paths resolved differently after upgrading to ASP.NET 4',
 'Materialize datepicker - Rendering when an icon is clicked',
 'Creating PyPi package - Could not find a version that satisfies the requirement iso8601',
 'How can I analyze a confusion matrix?']

Размер нескольких первых строк

In [24]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [25]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [26]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    vector = question_to_vec(question, embeddings, tokenizer).reshape(1,dim)
    result = [cosine_similarity(vector,question_to_vec(candidate, embeddings,tokenizer).reshape(1,dim)) for candidate in candidates]
    sort_result = np.argsort(result,axis=0)[::-1]
    result = [(sort_result[i][0][0],candidates[sort_result[i][0][0]]) for i in range(len(candidates))]
    return result

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [27]:
questions = ['converting string to list', 'Sending array via Ajax fails'] 

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],
              
              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [28]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
        print(ranks)
        print()

[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]

[(0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results'), (1, 'WPF- How to update the changes in list item of a list')]



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [29]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(0, 'Getting all list items of an unordered list in PHP'), #скрыт
            (2, 'select2 not displaying search results'), #скрыт
            (1, 'WPF- How to update the changes in list item of a list')]] #скрыт

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента? **021**(если учитывать, что в словаре lower-слова) / **102** (иначе)

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [30]:
from tqdm.notebook import tqdm

In [31]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

In [32]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

DCG@   1: 0.415 | Hits@   1: 0.415
DCG@   5: 0.502 | Hits@   5: 0.582
DCG@  10: 0.524 | Hits@  10: 0.650
DCG@ 100: 0.570 | Hits@ 100: 0.874
DCG@ 500: 0.583 | Hits@ 500: 0.973
DCG@1000: 0.586 | Hits@1000: 1.000



### Эмбеддинги, обученные на корпусе похожих вопросов

In [33]:
train_data = read_corpus('./data/train.tsv')

In [34]:
train_data[0]

['converting string to list',
 'Convert Google results object (pure js) to Python object\n']

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

In [35]:
train_data = [' '.join(element) for element in train_data]

In [38]:
words = [tokenizer.tokenize(element) for element in train_data]

In [39]:
window_len = max([len(words[i]) for i in range(len(words))]) #выбираем такой размер окна, чтобы учесть все слова в предложении

In [40]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(words, # data for model to train on
                 size=200,                 # embedding vector size
                 min_count=5,             # consider words that occured at least 5 times
                 window=window_len).wv

In [41]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

In [42]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

DCG@   1: 0.314 | Hits@   1: 0.314
DCG@   5: 0.388 | Hits@   5: 0.456
DCG@  10: 0.410 | Hits@  10: 0.527
DCG@ 100: 0.452 | Hits@ 100: 0.737
DCG@ 500: 0.475 | Hits@ 500: 0.916
DCG@1000: 0.484 | Hits@1000: 1.000



####ПОПРОБУЕМ ИЗМЕНИТЬ ПРИНЦИП ТОКЕНИЗАЦИИ:

In [43]:
from nltk.tokenize import WordPunctTokenizer
pt = WordPunctTokenizer()

In [44]:
words = [pt.tokenize(element) for element in train_data]

In [45]:
window_len = max([len(words[i]) for i in range(len(words))]) #выбираем такой размер окна, чтобы учесть все слова в предложении

In [46]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(words, # data for model to train on
                 size=200,                 # embedding vector size
                 min_count=5,             # consider words that occured at least 5 times
                 window=window_len).wv
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

DCG@   1: 0.312 | Hits@   1: 0.312
DCG@   5: 0.380 | Hits@   5: 0.444
DCG@  10: 0.406 | Hits@  10: 0.523
DCG@ 100: 0.449 | Hits@ 100: 0.740
DCG@ 500: 0.472 | Hits@ 500: 0.917
DCG@1000: 0.481 | Hits@1000: 1.000



####ПОПРОБУЕМ ЛЕММАТИЗАЦИЮ:

In [47]:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [48]:
words = [tokenizer.tokenize(element) for element in train_data]
words = [[lemmatizer.lemmatize(element) for element in words[i]] for i in range(len(words))]

In [ ]:
window_len = max([len(words[i]) for i in range(len(words))]) #выбираем такой размер окна, чтобы учесть все слова в предложении

In [49]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(words, # data for model to train on
                 size=200,                 # embedding vector size
                 min_count=5,             # consider words that occured at least 5 times
                 window=window_len).wv
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

DCG@   1: 0.289 | Hits@   1: 0.289
DCG@   5: 0.357 | Hits@   5: 0.421
DCG@  10: 0.382 | Hits@  10: 0.498
DCG@ 100: 0.422 | Hits@ 100: 0.692
DCG@ 500: 0.447 | Hits@ 500: 0.890
DCG@1000: 0.459 | Hits@1000: 1.000



####ПОПРОБУЕМ СТЕММИНГ:

In [50]:
import nltk
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()


In [51]:
words = [tokenizer.tokenize(element) for element in train_data]
words = [[porter_stemmer.stem(element) for element in words[i]] for i in range(len(words))]

In [52]:
window_len = max([len(words[i]) for i in range(len(words))]) #выбираем такой размер окна, чтобы учесть все слова в предложении

In [57]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(words, # data for model to train on
                 size=200,                 # embedding vector size
                 min_count=5,             # consider words that occured at least 5 times
                 window=window_len).wv
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

DCG@   1: 0.236 | Hits@   1: 0.236
DCG@   5: 0.311 | Hits@   5: 0.380
DCG@  10: 0.329 | Hits@  10: 0.435
DCG@ 100: 0.374 | Hits@ 100: 0.658
DCG@ 500: 0.401 | Hits@ 500: 0.868
DCG@1000: 0.415 | Hits@1000: 1.000



####Мой подход к решению:

Я буду приводить слова к нижнему регистру и исключать стоп-слова, а также те, которые состоят из одного символа.

In [58]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении
        
        return: векторное представление для вопроса
    """
    
    vector = [embeddings[word.lower()] for word in tokenizer.tokenize(question) if ((word.lower() in embeddings) and (word.lower() not in stop_words) and (len(word) != 1))]
    if (len(vector) == 0):
          vector = np.array([0]*dim)
          return vector
    return np.mean(vector, axis=0)

In [59]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words("english")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [60]:
words = [tokenizer.tokenize(element) for element in train_data]
words = [[element.lower() for element in words[i] if ((element.lower() not in stop_words) and len(element) != 1)] for i in range(len(words))]

In [61]:
window_len = max([len(words[i]) for i in range(len(words))]) #выбираем такой размер окна, чтобы учесть все слова в предложении

In [62]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(words, # data for model to train on
                 size=200,                 # embedding vector size
                 min_count=5,             # consider words that occured at least 5 times
                 window=window_len).wv
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

DCG@   1: 0.442 | Hits@   1: 0.442
DCG@   5: 0.547 | Hits@   5: 0.636
DCG@  10: 0.572 | Hits@  10: 0.715
DCG@ 100: 0.610 | Hits@ 100: 0.894
DCG@ 500: 0.620 | Hits@ 500: 0.976
DCG@1000: 0.623 | Hits@1000: 1.000



### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

## Вывод:


* Какой принцип токенизации даёт качество лучше и почему?

**Как мы можем видеть выше принцип токенезации, реализованный в классе MyTokenizer дает качество лучше, чем WordPunctTokenizer, так как он исключает все знаки, кроме буквенно-цифровых.**

* Помогает ли нормализация слов?

**Cтемминг и лемматизация не помогают в повышении качества.**

* Какие эмбеддинги лучше справляются с задачей и почему?

**Эмбеддинги, предобученные на постах Stack Overflow лучше справляются с задачей, потому что словарь валидный и оптимизированный.**

* Почему получилось плохое качество решения задачи?

**Плохое качество решения задачи получилось из-за наличия множества стоп-слов, а также различного регистра.**

* Предложите свой подход к решению задачи.

**Я буду приводить слова к нижнему регистру и исключать стоп-слова, а также те, которые состоят из одного символа. Я получил качество по DCG@1 - 0.442**

